# 주식 및 ETF 데이터 수집

In [2]:
# !pip install pykrx
# !pip install beautifulsoup4 pandas openpyxl selenium
# !pip install webdriver-manager

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from pykrx import stock
import datetime
import os


# 역대 종목 리스트 크롤링

In [1]:
# # 시작 연도와 현재 연도 설정
# start_year = 1990
# current_year = datetime.datetime.now().year

# # 상장 종목 리스트 수집
# all_tickers = set()

# for year in range(start_year, current_year + 1):
#     date_str = f"{year}1201"  # 해당 연도의 마지막 날짜

#     # 코스닥 시장 종목 수집
#     tickers_kosdaq = stock.get_market_ticker_list(date_str, market='KOSDAQ')
#     all_tickers.update(tickers_kosdaq)

#     # 코스피 시장 종목 수집
#     tickers_kospi = stock.get_market_ticker_list(date_str, market='KOSPI')
#     all_tickers.update(tickers_kospi)

#     # print(all_tickers)
#     time.sleep(1)


# 전체 종목 데이터 크롤링 

In [26]:
# import os

# # 저장할 폴더 경로 설정
# folder_path = 'korea_stock'  # 여기에 원하는 폴더 경로를 입력하세요.


# # 상장 폐지된 종목 포함한 모든 종목의 데이터 수집 및 저장
# for ticker in all_tickers:
#     # print(ticker)
#     # 티커의 마지막 자리가 0이 아니면 건너뜀
#     if ticker[-1] != '0':
#         continue
#     filename = os.path.join(folder_path, f"{ticker}.xlsx")
    
#    # 이미 파일이 존재하면 다음 티커로 넘어감
#     if os.path.exists(filename):
#         continue

#     df1 = stock.get_market_ohlcv("19800102", "20231225", ticker)
#     time.sleep(0.1)

#     # df1 또는 df2 중 하나라도 비어 있으면 다음 티커로 넘어감
#     if df1.empty:
#         continue

#     df2 = stock.get_market_fundamental("19800102", "20231225", ticker)
#     time.sleep(0.1)

#     # df2가 비어있으면 다음 티커로 넘어감
#     if df2.empty:
#         continue

#     df3 = stock.get_market_cap("19800102", "20231225", ticker)
#     time.sleep(0.1)

#     # 데이터 병합
#     combined_df = pd.concat([df1, df2, df3], axis=1, join='outer')
    
#     # 파일 저장 (Excel 형식)
#     combined_df.to_excel(filename)
#     time.sleep(1)
# print('done')

# 종목 데이터 크롤링 및 데이터 업데이트 동시

In [9]:

# 저장된 파일이 있는 폴더 경로
folder_path = '.'

# 현재 날짜 기준으로 최근 월 종목 리스트 수집
current_date = datetime.datetime.now()
last_month_str = current_date.strftime("%Y%m") + '01'

tickers_kosdaq = stock.get_market_ticker_list(last_month_str, market='KOSDAQ')
tickers_kospi = stock.get_market_ticker_list(last_month_str, market='KOSPI')
recent_tickers = set(tickers_kosdaq + tickers_kospi)

# 각 티커에 대한 처리
for ticker in recent_tickers:
    # 티커의 마지막 자리가 0이 아니면 건너뜀
    if ticker[-1] != '0':
        continue

    file_path = os.path.join(folder_path, f"{ticker}.xlsx")

    # 파일이 존재하지 않으면 새로 생성
    if not os.path.exists(file_path):
        df1 = stock.get_market_ohlcv("19800102", current_date.strftime("%Y%m%d"), ticker)
            # df1 또는 df2 중 하나라도 비어 있으면 다음 티커로 넘어감
        if df1.empty:
            continue
        df2 = stock.get_market_fundamental("19800102", current_date.strftime("%Y%m%d"), ticker)
        if df2.empty:
            continue
        df3 = stock.get_market_cap("19800102", current_date.strftime("%Y%m%d"), ticker)
        combined_df = pd.concat([df1, df2, df3], axis=1, join='outer')
        combined_df.to_excel(file_path)
        print(f"새 파일 생성: {ticker}")
    else:
        # 기존 파일 로드 및 업데이트
        existing_df = pd.read_excel(file_path, index_col=0)
        # 마지막 날짜 확인 및 새로운 데이터 수집
        last_date = existing_df.index[-1]
        next_date = (pd.to_datetime(last_date) + datetime.timedelta(days=1)).strftime("%Y%m%d")
        # 기존 데이터에서 중복을 피하기 위해 next_date 이후 데이터 삭제
        existing_df = existing_df[:last_date]
        new_ohlcv = stock.get_market_ohlcv(next_date, current_date.strftime("%Y%m%d"), ticker)
        new_fundamental = stock.get_market_fundamental(next_date, current_date.strftime("%Y%m%d"), ticker)
        new_cap = stock.get_market_cap(next_date, current_date.strftime("%Y%m%d"), ticker)

        
        new_combined_df = pd.concat([new_ohlcv, new_fundamental, new_cap], axis=1, join='outer')
          # 열 이름과 순서 일치시키기 전에 열의 수를 확인
        if len(new_combined_df.columns) != len(existing_df.columns):
            # print(f"열의 수가 일치하지 않음: {ticker}")
            # print(new_combined_df.columns)
            continue
        # 열 이름과 순서 일치시키기
        new_combined_df.columns = existing_df.columns
        
        
        
        updated_df = pd.concat([existing_df, new_combined_df],axis=0)
        updated_df.to_excel(file_path)
        print(f"{ticker} 업데이트 완료")

새 파일 생성: 025750
새 파일 생성: 014710
새 파일 생성: 355690


KeyboardInterrupt: 

In [5]:
# PER 컬럼과 '고가' 컬럼이 없는 파일을 찾아서 삭제
for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        ticker = file.replace('.xlsx', '')
        if ticker not in recent_tickers:
            file_path = os.path.join(folder_path, file)
            df = pd.read_excel(file_path, index_col=0)
            if 'PER' not in df.columns or '고가' not in df.columns:
                os.remove(file_path)
                print(f"{file} 파일 삭제됨")

etf_list.xlsx 파일 삭제됨


In [3]:
# import pandas as pd
# import os

# # 조건 설정
# per_threshold = 10
# pbr_threshold = 1
# div_threshold = 3

# # 저장된 파일이 있는 폴더 경로
# folder_path = 'korea_stock'

# for file in os.listdir(folder_path):
#     if file.endswith(".xlsx"):
#         file_path = os.path.join(folder_path, file)
#         df = pd.read_excel(file_path, index_col=0)

#         # 조건에 맞는 데이터가 있는지 확인
#         condition = (df['PER'] > 0) & (df['PER'] <= per_threshold) & \
#                     (df['PBR'] > 0) & (df['PBR'] <= pbr_threshold) & \
#                     (df['DIV'] >= div_threshold)

#         if not df[condition].empty:
#             continue  # 조건을 만족하는 데이터가 있으면 다음 파일로 넘어감

#         # 조건을 만족하는 데이터가 없으면 파일 삭제
#         os.remove(file_path)
#         print(f"{file} 파일 삭제됨")


000040.xlsx 파일 삭제됨
000050.xlsx 파일 삭제됨
000080.xlsx 파일 삭제됨
000100.xlsx 파일 삭제됨
000120.xlsx 파일 삭제됨
000300.xlsx 파일 삭제됨
000420.xlsx 파일 삭제됨
000540.xlsx 파일 삭제됨
000610.xlsx 파일 삭제됨
000650.xlsx 파일 삭제됨
000660.xlsx 파일 삭제됨
000670.xlsx 파일 삭제됨
000680.xlsx 파일 삭제됨
000720.xlsx 파일 삭제됨
000760.xlsx 파일 삭제됨
000790.xlsx 파일 삭제됨
000830.xlsx 파일 삭제됨
001000.xlsx 파일 삭제됨
001070.xlsx 파일 삭제됨
001140.xlsx 파일 삭제됨
001250.xlsx 파일 삭제됨
001260.xlsx 파일 삭제됨
001360.xlsx 파일 삭제됨
001380.xlsx 파일 삭제됨
001460.xlsx 파일 삭제됨
001510.xlsx 파일 삭제됨
001550.xlsx 파일 삭제됨



KeyboardInterrupt



In [3]:
import pandas as pd
import os

# 조건 설정
per_threshold = 10
pbr_threshold = 1
div_threshold = 3

# 저장된 파일이 있는 폴더 경로
folder_path = 'korea_stock'

for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_excel(file_path, index_col=0)
    
            # 데이터 타입을 float로 변환
            for column in ['PER', 'PBR', 'DIV']:
                if column in df.columns:
                    df[column] = pd.to_numeric(df[column], errors='coerce')
    
            # 조건 확인
            condition = (df['PER'] > 0) & (df['PER'] <= per_threshold) & \
                        (df['PBR'] > 0) & (df['PBR'] <= pbr_threshold) & \
                        (df['DIV'] >= div_threshold)
    
            if not df[condition].empty:
                continue  # 조건을 만족하는 데이터가 있으면 다음 파일로 넘어감
    
            # 조건을 만족하지 않는 경우 어떤 조건이 만족되지 않았는지 확인
            if 'PER' not in df.columns or df['PER'].isnull().all() or (df['PER'] <= 0).all() or (df['PER'] > per_threshold).all():
                reason = 'PER'
            elif 'PBR' not in df.columns or df['PBR'].isnull().all() or (df['PBR'] <= 0).all() or (df['PBR'] > pbr_threshold).all():
                reason = 'PBR'
            elif 'DIV' not in df.columns or df['DIV'].isnull().all() or (df['DIV'] < div_threshold).all():
                reason = 'DIV'
            if condition.any():
                continue  # 조건을 만족하는 데이터가 하나라도 있으면 다음 파일로 넘어감
    
            print(f"{file} 파일 삭제됨 (조건 불만족: {reason})")
            os.remove(file_path)
        except Exception as e:
            print(f"파일 읽기 실패: {file_path}, 오류: {e}")
            continue

파일 읽기 실패: korea_stock\000040.xlsx, 오류: name 'reason' is not defined
파일 읽기 실패: korea_stock\000050.xlsx, 오류: name 'reason' is not defined
파일 읽기 실패: korea_stock\000080.xlsx, 오류: name 'reason' is not defined
파일 읽기 실패: korea_stock\000100.xlsx, 오류: name 'reason' is not defined
파일 읽기 실패: korea_stock\000120.xlsx, 오류: name 'reason' is not defined
파일 읽기 실패: korea_stock\000300.xlsx, 오류: name 'reason' is not defined
파일 읽기 실패: korea_stock\000540.xlsx, 오류: name 'reason' is not defined
파일 읽기 실패: korea_stock\000650.xlsx, 오류: name 'reason' is not defined
000660.xlsx 파일 삭제됨 (조건 불만족: DIV)
000670.xlsx 파일 삭제됨 (조건 불만족: DIV)
000680.xlsx 파일 삭제됨 (조건 불만족: DIV)
000720.xlsx 파일 삭제됨 (조건 불만족: DIV)
000760.xlsx 파일 삭제됨 (조건 불만족: DIV)
001000.xlsx 파일 삭제됨 (조건 불만족: DIV)
001070.xlsx 파일 삭제됨 (조건 불만족: DIV)
001140.xlsx 파일 삭제됨 (조건 불만족: DIV)
001250.xlsx 파일 삭제됨 (조건 불만족: DIV)
001260.xlsx 파일 삭제됨 (조건 불만족: DIV)
001360.xlsx 파일 삭제됨 (조건 불만족: DIV)
001380.xlsx 파일 삭제됨 (조건 불만족: DIV)
001460.xlsx 파일 삭제됨 (조건 불만족: DIV)
001510.xlsx 파일 삭제됨 (조건 불만족:

## ETF 티커 & 리스트 생성  

In [6]:
# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')  # 브라우저 창을 띄우지 않음
options.add_argument('disable-gpu')  # GPU 가속 비활성화
options.add_argument('lang=ko_KR')  # 한국어 페이지

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [7]:
# URL에서 HTML 컨텐츠를 가져옴
url = 'https://finance.naver.com/sise/etf.naver'
driver.get(url)


In [8]:

# 종목명과 티커를 추출하여 리스트로 저장
etf_data = []
rows = driver.find_elements(By.CSS_SELECTOR, '#etfItemTable > tr')
for row in rows:
    name_col = row.find_elements(By.CSS_SELECTOR, 'td.ctg a')
    if name_col:
        name = name_col[0].text.strip()
        ticker = name_col[0].get_attribute('href').split('=')[-1]  # 종목명 링크에서 티커 추출
        etf_data.append([name, ticker])

# 드라이버 종료
driver.quit()

# DataFrame 생성
df = pd.DataFrame(etf_data, columns=['종목명', '티커'])

# DataFrame을 Excel 파일로 저장
excel_filename = 'etf_list.xlsx'
df.to_excel(excel_filename, index=False)

print(f"Excel 파일로 저장되었습니다: {excel_filename}")


Excel 파일로 저장되었습니다: etf_list.xlsx
